In [1]:
from pyld import jsonld
import json

In [2]:
jsonld_input = ''
with open('./robot_arm_simulator/lamp_thing_with_defaults.jsonld', 'r') as f:
    jsonld_input = json.load(f)

In [3]:
print(jsonld_input)
#print(json.dumps(jsonld.expand(jsonld_input), indent=2))

{'@context': 'https://www.w3.org/2019/wot/td/v1', 'id': 'urn:dev:ops:32473-WoTLamp-1234', 'title': 'MyLampThing', 'securityDefinitions': {'basic_sc': {'scheme': 'basic', 'in': 'header'}}, 'security': ['basic_sc'], 'properties': {'status': {'type': 'string', 'readOnly': False, 'writeOnly': False, 'forms': [{'op': ['readproperty', 'writeproperty'], 'href': 'https://mylamp.example.com/status', 'contentType': 'application/json'}]}}, 'actions': {'toggle': {'safe': False, 'idempotent': False, 'forms': [{'op': 'invokeaction', 'href': 'https://mylamp.example.com/toggle', 'contentType': 'application/json'}]}}, 'events': {'overheating': {'data': {'type': 'string', 'readOnly': False, 'writeOnly': False}, 'forms': [{'op': 'subscribeevent', 'href': 'https://mylamp.example.com/oh', 'contentType': 'application/json', 'subprotocol': 'longpoll'}]}}}


In [4]:
def extract_elements(jsonld_input, key):
    return jsonld_input.get(key)

def extract_actions(jsonld_input):
    return extract_elements(jsonld_input, 'actions')

def extract_events(jsonld_input):
    return extract_elements(jsonld_input, 'events')

def extract_properties(jsonld_input):
    return extract_elements(jsonld_input, 'properties')

In [5]:
actions = extract_actions(jsonld_input)
events = extract_events(jsonld_input)
properties = extract_properties(jsonld_input)

In [6]:
print(actions)

{'toggle': {'safe': False, 'idempotent': False, 'forms': [{'op': 'invokeaction', 'href': 'https://mylamp.example.com/toggle', 'contentType': 'application/json'}]}}


In [7]:
class Http_Form:
    def __init__(self, name, jsonld_description, isAction=False):            
        print(jsonld_description)
        self.name = name
        self.endpoint = jsonld_description.get('href', '')
        self.content_type = jsonld_description.get('contentType', '')
        self.http_methods = []
        if 'htv:methodName' in jsonld_description:
            self.http_methods.append(jsonld_description.get('htv:methodName'))
        elif isAction:
            self.http_methods.append('POST') #default for action acc. to w3c
        elif 'op' in jsonld_description:
            # default for these operations according to w3c standard
            ops = jsonld_description.get('op')
            if type(ops) is str:
                ops = [ops]
            for op in ops:
                if op in {'readproperty', 'readallproperties', 'readmultipleproperties'}:
                    self.http_methods.append('GET')
                elif op in {'writeproperty', 'writeallproperties', 'writemultipleproperties'}:
                    self.http_methods.append('PUT')
                elif op == 'invokeaction':
                    self.http_methods.append('POST')
                elif op in {'subscribeevent', 'unsubscribeevent'}:
                    print(jsonld_description)
                    if 'subprotocol' in jsonld_description:
                        self.http_methods.append(jsonld_description.get('subprotocol'))
                    else:
                        print("ERROR: httpmethod of subscribeevent and unsubscribeevent not clearly defined")
                else:
                    print("ERROR: no http method assigned for op " + op)
                        
    def __str__(self):
        res = "Http_Form '" + self.name + "'\nendpoint: " + self.endpoint + "\ncontent_type: " + self.content_type + "\nhttp_methods: "
        for h in self.http_methods:
            res += h + ', '
        return  res
    

In [8]:
http_forms = []
for action_name, a in actions.items():
    for form in a.get('forms'):
        http_forms.append(('action', Http_Form(action_name, form, isAction=True)))

for property_name, p in properties.items():
    for form in p.get('forms'):
        http_forms.append(('property', Http_Form(property_name, form)))

for event_name, e in events.items():
    for form in e.get('forms'):
        http_forms.append(('event', Http_Form(event_name, form)))
for name, f in http_forms:
    print(name + " " + str(f))

{'op': 'invokeaction', 'href': 'https://mylamp.example.com/toggle', 'contentType': 'application/json'}
{'op': ['readproperty', 'writeproperty'], 'href': 'https://mylamp.example.com/status', 'contentType': 'application/json'}
{'op': 'subscribeevent', 'href': 'https://mylamp.example.com/oh', 'contentType': 'application/json', 'subprotocol': 'longpoll'}
{'op': 'subscribeevent', 'href': 'https://mylamp.example.com/oh', 'contentType': 'application/json', 'subprotocol': 'longpoll'}
action Http_Form 'toggle'
endpoint: https://mylamp.example.com/toggle
content_type: application/json
http_methods: POST, 
property Http_Form 'status'
endpoint: https://mylamp.example.com/status
content_type: application/json
http_methods: GET, PUT, 
event Http_Form 'overheating'
endpoint: https://mylamp.example.com/oh
content_type: application/json
http_methods: longpoll, 


In [9]:
ui_outputs = {'general_data_output',
               'ordered_domain_output',
               'fixed_range_ordered_domain_output'
             }
ui_inputs = {'stateless_trigger_button',
               'stateless_forward_backward_buttons',
               'general_data_input',
               'ordered_domain_input',
               'ordered_domain_with_neutral_value_input',
               'operating_mode_input',
               'ordered_domain_fixed_range_input',
               'boolean_switch_input',
               'set_position_input',
               'move_input'
            }

In [10]:
# TODO: parse data types and together with Http_Form data, determine corresponding UI element
# property: type; if type==object, then it's nested!
# action: (input & output)->type; if type==object, then it's nested!
# event: data->type (this is the type that is being received)

In [13]:
def parse_uri_variables(jsonld):
    '''
        part of InteractionAffordance, same level as forms and connected to forms (no uriVariables without forms)
        
        e.g.
        
        "uriVariables": {
                "p" : { "type": "integer", "minimum": 0, "maximum": 16, "@type": "eg:SomeKindOfAngle" },
                "d" : { "type": "integer", "minimum": 0, "maximum": 1, "@type": "eg:Direction" }
        },
        "forms": [{
          "href" : "http://192.168.1.25/left{?p,d}",
          "htv:methodName": "GET"
        }]
    '''
    
    # not essential for now. TODO: implement later
    
def parse_array_type(jsonld):
    '''
        "type": "array",
        "items" : {
            "type" : "number",
            "minimum": 0,
            "maximum": 255
        },
        "minItems": 3,
        "maxItems": 3
    '''

# for both integer and number
def parse_number_type(jsonld):
    '''
        "type": "number",
        "minimum": 0.0,
        "maximum": 100.0
    '''

def parse_object_type(jsonld):
    # recursively parse types
    '''
            "type": "object",
            "properties": {
                "from": {
                    "type": "integer",
                    "minimum": 0,
                    "maximum": 100
                },
                "to": {
                    "type": "integer",
                    "minimum": 0,
                    "maximum": 100
                },
                "duration": {"type": "number"}
            },
            "required": ["to","duration"],
    '''
    
def parse_type(jsonld):
    t = jsonld.get('type')
    # TODO: check for enum
    
def parse_actions(actions, http_forms):
    '''
        "input": {
            "type": "object",
            "properties": {
                "from": {
                    "type": "integer",
                    "minimum": 0,
                    "maximum": 100
                },
                "to": {
                    "type": "integer",
                    "minimum": 0,
                    "maximum": 100
                },
                "duration": {"type": "number"}
            },
            "required": ["to","duration"],
        },
        "output": {"type": "string"},
    '''
    for action_name, a in actions.items():
        input = []
        output = []
        
        for form in a.get('forms'):
            http_forms.append(('action', Http_Form(action_name, form, isAction=True)
                               
        '''
            Heuristics for actions:
                - no input -> simple trigger
                - input -> - single input -> single UI_input_element (no trigger button, interaction with UI_input_element automatically triggers action, except for general (string) input)
                           - multiple inputs -> for each input UI_input_element and trigger button to trigger action
                - output -> sensor outputs; or could it also render back into a stateful actuator? 
                                            RE: might make sense for stateful actuators, i.e. request to change state
                                            to 10 and actuator does not change. Then input trigger-wheel will turn
                                            back to actual value. In the general case the output seems to be more something
                                            like a feedback. i.e. "success" or "error"
                                            
        '''
            
    
def parse_properties(properties, http_forms):
    '''
        According to w3c standard: "Property instances are also instances of the class DataSchema. 
        Therefore, it can contain the type, unit, readOnly and writeOnly members, among others."
    '''
    for property_name, p in properties.items():
        for form in p.get('forms'):
            http_forms.append(('property', Http_Form(property_name, form)))
        property_type = p.get('type')
        if property_type in {'array', 'string', 'number', 'integer', 'boolean'}:
            # TODO: simple type, infer UI element for each form?
        elif property_type ==  'object':
            # TODO: recursive type checking
        elif prperty_type == 'null':
            # TODO: define what happens here
            
def parse_events(events, http_forms):
    '''
        subscription (optional): Defines data that needs to be passed upon subscription.
        data (optional): Defines the data schema of the Event instance messages pushed by the Thing.
        cancellation (optional): Defines any data that needs to be passed to cancel a subscription
        
        e.g.
        
        "subscription": {
            "type": "object",
            "properties": {
                "callbackURL": {
                    "type": "string",
                    "format": "uri",
                    "description": "Callback URL provided by subscriber for Webhook notifications.",
                    "writeOnly": true
                },
                "subscriptionID": {
                    "type": "string",
                    "description": "Unique subscription ID for cancellation provided by WebhookThing.",
                    "readOnly": true
                }
            }
        },
        "data": {
            "type": "number",
            "description": "Latest temperature value that is sent to the callback URL."
        },
        "cancellation": {
            "type": "object",
            "properties": {
                "subscriptionID": {
                    "type": "integer",
                    "description": "Required subscription ID to cancel subscription.",
                    "writeOnly": true
                }
            }
        },
    '''
    for event_name, e in events.items():
        for form in e.get('forms'):
            http_forms.append(('event', Http_Form(event_name, form)))
        event_output_type = e.get('data').get('type')
        if property_type in {'array', 'string', 'number', 'integer', 'boolean'}:
            # TODO: simple type, infer UI element for each form?
        elif property_type ==  'object':
            # TODO: recursive type checking
        elif prperty_type == 'null':
            # TODO: define what happens here
        

IndentationError: expected an indented block (<ipython-input-13-1947a882791a>, line 48)